In [ ]:
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.3/730.3 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch.optim import Adam
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline
from pykeen.nn.representation import Representation
from pykeen.models import ComplEx, RotatE, TransE
from pykeen.evaluation import RankBasedEvaluator
from pykeen.sampling import BasicNegativeSampler

In [ ]:
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
df = pd.read_csv("/content/triples.tsv", sep="\t", header=None, names=["head", "relation", "tail"])
entities = set(df["head"]).union(set(df["tail"]))

In [ ]:
entity_to_embedding = {}

In [ ]:
with torch.no_grad():
    for entity in tqdm(entities):
        inputs = tokenizer(entity, return_tensors="pt", truncation=True, padding=True).to(device)
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze()
        entity_to_embedding[entity] = embedding.squeeze(0)

100%|██████████| 82586/82586 [07:07<00:00, 193.19it/s]


In [ ]:
tf = TriplesFactory.from_path("triples.tsv", delimiter="\t")

In [ ]:
class PrecomputedEmbedding(Representation):
    def __init__(self, entity_tensor: torch.FloatTensor, trainable: bool = False):
        num_entities, embedding_dim = entity_tensor.shape
        super().__init__(max_id=num_entities, shape=(embedding_dim,))
        self.embeddings = torch.nn.Parameter(entity_tensor, requires_grad=trainable)

    def _plain_forward(self, indices: torch.LongTensor) -> torch.FloatTensor:
        return self.embeddings[indices]

In [ ]:
entity_tensor = torch.stack([entity_to_embedding[entity] for entity in tf.entity_to_id])
embedder = PrecomputedEmbedding(entity_tensor, trainable=False)

INFO:pykeen.nn.representation:Inferred unique=True for PrecomputedEmbedding()


In [ ]:
train_factory, valid_factory, test_factory = tf.split([0.8, 0.1, 0.1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [380469, 57314, 57315]


In [ ]:
model = RotatE(triples_factory=tf, embedding_dim=384)

INFO:pykeen.nn.representation:Inferred unique=False for Embedding()
INFO:pykeen.nn.representation:Inferred unique=False for Embedding()


In [ ]:
optimizer = Adam(params=model.get_grad_params(), lr=0.0001)

In [ ]:
result = pipeline(
    training=train_factory,
    validation=valid_factory,
    testing=test_factory,
    model=model,
    model_kwargs=dict(
        entity_representations=embedder,
    ),
    training_kwargs=dict(
        num_epochs=500,
        batch_size=2048,
    ),
    evaluator_kwargs=dict(
        batch_size=2048,
    ),
)

print(f"Hits@1: {result.metric_results.get_metric('hits_at_1'):.4f}")
print(f"Hits@3: {result.metric_results.get_metric('hits_at_3'):.4f}")
print(f"Hits@10: {result.metric_results.get_metric('hits_at_10'):.4f}")
print(f"MRR: {result.metric_results.get_metric('mrr'):.4f}")
print(f"MR: {result.metric_results.get_metric('mr'):.4f}")

INFO:pykeen.pipeline.api:Using device: None


Training epochs on cuda:0:   0%|          | 0/500 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0.00/224 [00:00<?, ?batch/s]